# Prepare things

## Install requirements

In [1]:
!pip install underthesea
!pip install emoji

     |████████████████████████████████| 7.5MB 7.6MB/s 
     |████████████████████████████████| 1.3MB 54.0MB/s 
     |████████████████████████████████| 747kB 53.2MB/s 
     |████████████████████████████████| 245kB 51.1MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 753.2MB 19kB/s 
     |████████████████████████████████| 901kB 55.4MB/s 
     |████████████████████████████████| 1.1MB 46.6MB/s 
     |████████████████████████████████| 2.9MB 39.6MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=5d13fb3cef4415c44749a4ec7dc0e7845a60982fa49af894a1e1b9b72dfb538e
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.5.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.1 which is incompatib

## Mount to google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ls '/content/gdrive/My Drive/Colab Notebooks/dataset/'

Mounted at /content/gdrive
train_dataset.xlsx


# IMPORTANT! Read file into dataframe

*   Check the Google Drive path which contains the dataset file.
*   Input the correct file name, also the column names which are used for comment and sentiment.



In [3]:
path = "/content/gdrive/My Drive/Colab Notebooks"

FILE_NAME = "train_dataset.xlsx"
COMMENT_COLUMN_NAME = "comment"
SENTIMENT_COLUMN_NAME = "sentiment"

In [4]:
import pandas as pd

df = pd.read_excel(f'{path}/dataset/{FILE_NAME}', usecols=["id", COMMENT_COLUMN_NAME, SENTIMENT_COLUMN_NAME])
df.head()

,id,comment,sentiment
0,2,"Bài của Phan Mạnh Quỳnh qá hay xuất sắc, hẹn b...",Tích cực
1,3,"Sao Cha Không? Hay quá A Xìn ơi, A Quỳnh ơi",Tích cực
2,7,giọng anh Quuỳnh cất lên là bào cảm xúc dâng t...,Tích cực
3,8,Tôi xem đi xem lại trailer này để nghe Phan Mạ...,Tích cực
4,9,Phan Mạnh Quỳnh cất tiếng hát lên một cái thôi...,Tích cực


# Implement help functions


 

## Normalized function for some specific cases in vietnamese

In [5]:
import re

slang_dict = {"bình thường": ["bth","bt","bthg"], "không":["k","K","ko","k0","kO","Ko","khong","kh","khg","hok","hk"], 
              "rồi":["r", "R"], "anh":["a","A"], "mình":["mk","Mk"], "vậy":["z","Z","v","V"], "gì":["j","J","ji"],"trước":["trc"],
              "đi":["ik"], "mọi người": ["mng","Mng","mn","Mn"],"giáo viên":["gv"],"thích":["thix"], "được":["đc","Đc","dc"]}
celeb_dict = {"Trấn Thành": ["xìn","Xìn","TT","tt","Tran Thanh","Tran thanh"], 
              "Phan Mạnh Quỳnh": ["pmq", "PMQ", "phan manh quynh"]}
en_dict = {"trailer": ["traler"], "porn":["pỏn","Pỏn"], "free":["fre"], "full":["ful"], "facebook":["facebok"]}

def create_norm_dict(input_dict):
  norm = {}
  for key, values in input_dict.items():
    for value in values:
      norm[value] = key
  return norm

def normalize(text, words):
  regex = r"\b(?:" + "|".join(re.escape(word) for word in words) + r")\b"
  reobj = re.compile(regex, re.I)
  try:
    res = reobj.sub(lambda x:words[x.group(0)], text)
  except Exception as e:
    res = text
  return res

bag_dicts = {}
for d in (slang_dict,celeb_dict,en_dict): 
  bag_dicts.update(d)
norm_dict = create_norm_dict(bag_dicts)
print(norm_dict)

{'bth': 'bình thường', 'bt': 'bình thường', 'bthg': 'bình thường', 'k': 'không', 'K': 'không', 'ko': 'không', 'k0': 'không', 'kO': 'không', 'Ko': 'không', 'khong': 'không', 'kh': 'không', 'khg': 'không', 'hok': 'không', 'hk': 'không', 'r': 'rồi', 'R': 'rồi', 'a': 'anh', 'A': 'anh', 'mk': 'mình', 'Mk': 'mình', 'z': 'vậy', 'Z': 'vậy', 'v': 'vậy', 'V': 'vậy', 'j': 'gì', 'J': 'gì', 'ji': 'gì', 'trc': 'trước', 'ik': 'đi', 'mng': 'mọi người', 'Mng': 'mọi người', 'mn': 'mọi người', 'Mn': 'mọi người', 'gv': 'giáo viên', 'thix': 'thích', 'đc': 'được', 'Đc': 'được', 'dc': 'được', 'xìn': 'Trấn Thành', 'Xìn': 'Trấn Thành', 'TT': 'Trấn Thành', 'tt': 'Trấn Thành', 'Tran Thanh': 'Trấn Thành', 'Tran thanh': 'Trấn Thành', 'pmq': 'Phan Mạnh Quỳnh', 'PMQ': 'Phan Mạnh Quỳnh', 'phan manh quynh': 'Phan Mạnh Quỳnh', 'traler': 'trailer', 'pỏn': 'porn', 'Pỏn': 'porn', 'fre': 'free', 'ful': 'full', 'facebok': 'facebook'}


## Split and Demoji into normal words

In [6]:
import emoji
import functools
import operator

def split_demoji(text):
  em_split_emoji = emoji.get_emoji_regexp().split(text)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  em_split = functools.reduce(operator.concat, em_split_whitespace)
  return emoji.demojize(' '. join(em_split))

# test
# split_demoji(df["comment"][215])

## Remove punctuations and special characters with Regex

In [7]:
def remove_punc(text):
  text = text.replace('\n', ' ')
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,“”'‘’]", ' ', text)
  return text

# Preprocess with the following steps

*   Step 1: Replace all links (http[s]://...) and timestamp (1:05, 2:08)
*   Step 2: Normalize text for some specific cases (ko, k --> không, bth, bt --> bình thường, ...)
*   Step 3: Split and demoji (❤️❤️❤️ --split into--> ❤️ ❤️ ❤️ --demoji--> :heart: :heart: :heart:)
*   Step 4: Remove continuous duplicate words within text (hay quá điiiiiiiii)
*   Step 5: Remove punctuations and special characters
*   Step 6: Tokenize words




In [8]:
import itertools
from underthesea import word_tokenize

tokenize_list = []

def preprocess(text):
  # Replace all links and timestamp
  text = re.sub('http[s]?://\S+', '', text)
  text = re.sub(r"([\d{1,2}:\d{2}(:\d{2})?])", ' ', text)

  # Normalize
  text = normalize(text, norm_dict)

  # Demojize
  text = split_demoji(text)

  # Process continuous duplicate words within text
  text = ''.join(i for i, _ in itertools.groupby(text))

  # Remove punctuations and special characters
  text = remove_punc(text)

  # word tokenize
  tokens = word_tokenize(text)
  tokenize_list.append(tokens)

  return ' '.join(tokens).lower()

# test preprocess
# test_text = df["comment"][210]
# preprocess_text(test_text)

In [9]:
import numpy as np

processed_text = []
for comment in df["comment"]:
  processed_text.append(preprocess(comment))
  
df['tokenize'] = np.array(tokenize_list)
df['preprocessed'] = np.array(processed_text)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


,id,comment,sentiment,tokenize,preprocessed
0,2,"Bài của Phan Mạnh Quỳnh qá hay xuất sắc, hẹn b...",Tích cực,"[Bài, của, Phan Mạnh Quỳnh, qá, hay, xuất sắc,...",bài của phan mạnh quỳnh qá hay xuất sắc hẹn bố...
1,3,"Sao Cha Không? Hay quá A Xìn ơi, A Quỳnh ơi",Tích cực,"[Sao, Cha, Không, Hay, quá, anh, Trấn Thành, ơ...",sao cha không hay quá anh trấn thành ơi anh qu...
2,7,giọng anh Quuỳnh cất lên là bào cảm xúc dâng t...,Tích cực,"[giọng, anh, Quỳnh, cất, lên, là, bào, cảm xúc...",giọng anh quỳnh cất lên là bào cảm xúc dâng tr...
3,8,Tôi xem đi xem lại trailer này để nghe Phan Mạ...,Tích cực,"[Tôi, xem, đi, xem, lại, trailer, này, để, ngh...",tôi xem đi xem lại trailer này để nghe phan mạ...
4,9,Phan Mạnh Quỳnh cất tiếng hát lên một cái thôi...,Tích cực,"[Phan Mạnh Quỳnh, cất tiếng, hát, lên, một, cá...",phan mạnh quỳnh cất tiếng hát lên một cái thôi...


# Run Logistic Regression with Pipelines




In [10]:
import unidecode
from sklearn.base import BaseEstimator, TransformerMixin

class RemoveTone(BaseEstimator, TransformerMixin):
    def remove_tone(self, s):
        return unidecode.unidecode(s)

    def transform(self, x):
        return [self.remove_tone(s) for s in x]

    def fit(self, x, y=None):
        return self

In [15]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(df["preprocessed"], df[SENTIMENT_COLUMN_NAME], test_size=0.2, random_state=42)
pipe = Pipeline(steps=[
            ('features', FeatureUnion([
                ('lower_pipe', Pipeline([
                    ('tfidf', TfidfVectorizer(ngram_range=(1, 4), min_df=2))])),
                ('with_tone_char', TfidfVectorizer(ngram_range=(1, 6), min_df=2, analyzer='char')),
                ('remove_tone', Pipeline([
                    ('remove_tone', RemoveTone()),
                    ('tfidf', TfidfVectorizer(ngram_range=(1, 4), min_df=2))])),
            ])),
            ('estimator', LogisticRegression()),
])

pipe.fit(X_train, y_train)

# Predict on train & test
pred_train = pipe.predict(X_train)
pred_test = pipe.predict(X_test)

# Evaluate on train & test
print("Accuracy on train", accuracy_score(y_train, pred_train))
print("Accuracy on test", accuracy_score(y_test, pred_test))

print("F1-score on train", f1_score(y_train, pred_train, average=None))
print("F1-score on test", f1_score(y_test, pred_test, average=None))

Accuracy on train 0.9962859795728877
Accuracy on test 0.7888888888888889
F1-score on train [0.99579832 0.99617834 0.99664054]
F1-score on test [0.73684211 0.76237624 0.83928571]


# Compare and Export results on Train & Test

In [ ]:
import os

output_path = f"{path}/output"
if not os.path.exists(output_path):
    os.makedirs(output_path)

df.to_excel(f"{output_path}/logistic_processed.xlsx", sheet_name='processed')
print(f"Please find the output files in path={output_path}")

In [ ]:
def create_export_df(comment, sentiment, pred):
  export_df = pd.DataFrame()
  export_df[COMMENT_COLUMN_NAME]=np.array(comment)
  export_df[SENTIMENT_COLUMN_NAME]=np.array(sentiment)
  export_df['pred']=np.array(pred)
  export_df["wrong_pred"] = np.where(export_df[SENTIMENT_COLUMN_NAME] != export_df['pred'], True, False)
  return export_df

In [ ]:
train_df = create_export_df(X_train, y_train, pred_train)
train_df.to_excel(f"{output_path}/logistic_train.xlsx", sheet_name='train')

train_df[train_df["wrong_pred"]==True]

In [ ]:
test_df = create_export_df(X_test, y_test, pred_test)
test_df.to_excel(f"{output_path}/logistic_test.xlsx", sheet_name='test')

test_df[test_df["wrong_pred"]==True]

# Issue notes
- normalize dấu mũ
- boost thêm những từ quan trọng
- should remove celeb name? -> phim này khen nhưng phim khác chê -> tùy vào lượng cmt mà ra kết quả
- xem phim xong trích câu trong truyện -> label tích cực?
- một vài cmt tiếng anh ko đủ data để học